## retriever 만들기

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../.env', override=True)

apikey = os.getenv("OPENAI_API_KEY")


#### 문서 로더

파일
* TextLoader
| 일반 텍스트 파일(.txt) 읽기
* PDFPlumberLoader / PyPDFLoader
| PDF 문서 불러오기
* UnstructuredFileLoader
| 다양한 파일 포맷 (Word, PPT, PDF 등)을 구조 없이 불러오기
* CSVLoader
| CSV 파일의 각 행을 문서로 변환
* JSONLoader
| JSON 파일을 읽고 문서로 구성


웹/URL
* WebBaseLoader
| 일반 웹 페이지 크롤링
* SitemapLoader
| 사이트맵 기반으로 다수의 웹 페이지 로딩


문서 플랫폼
* NotionDBLoader
| Notion의 DB에서 문서 불러오기 (API 필요)
* ConfluenceLoader
| Atlassian Confluence 문서 불러오기


클라우드 문서
* GoogleDriveLoader
| 구글 드라이브에서 문서 불러오기
* OneDriveLoader
| 마이크로소프트 OneDrive에서 불러오기


코드/로컬
* GitLoader
| Git 저장소 내 파일 불러오기
* DirectoryLoader
| 특정 폴더 내 모든 문서 일괄 불러오기


메일
* OutlookMessageLoader
| Outlook .msg 파일 불러오기
* EmailLoader
| EML 또는 MIME 형식 이메일 로드


이미지/스캔
* UnstructuredImageLoader
| 이미지에서 텍스트 추출 (OCR 기반)
* UnstructuredPDFLoader
| PDF 전처리 고급

#### 새로나온 텍스트 스플릿 기능 (추후 적용 해보기)

* 유사도 기반 스플릿
* https://python.langchain.com/api_reference/experimental/text_splitter.html



In [53]:
# 문서 스플릿

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                            chunk_overlap=100)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("..\\ref\\docs\\law_test.txt")
# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1)

10

In [71]:
# 임베딩

import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 임시 DB 생성
# db = Chroma.from_documents(
#     documents=split_doc1, embedding=OpenAIEmbeddings(), 
#     collection_name="my_db"
# )

# 저장할 경로 지정
DB_PATH = ".\\chroma_db"

# # 문서를 디스크에 저장. 저장시 persist_directory에 저장할 경로를 지정.
# persist_db = Chroma.from_documents(
#     documents=split_doc1,
#     embedding=embeddings, 
#     persist_directory=DB_PATH, 
#     collection_name="my_db2"
# )

#디스크에서 문서를 로드.
db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="my_db2",
)

In [10]:
# 저장된 데이터 확인
persist_db.get()

{'ids': ['2bf19298-80a4-4289-a4f1-44bfa430cd8e',
  'da9243fb-957f-46aa-8e4d-8267c4bec91a',
  '179a459d-6ba6-47d4-a1c4-40b7c05a85fa',
  '1202f34f-2c31-40ee-bd50-f5e009da3126',
  '0f0cb2d1-fa41-47f0-8754-f94043847d55',
  'f749d566-25d4-4561-8100-102c74225744',
  '9ca95306-8b48-4005-a07f-497a8e6b80bb',
  '48337bd7-8b37-4b5e-9fb0-280f4006835b',
  '1ae6bf9a-8d5a-4f02-b242-c623c49ddd1b',
  '1207b315-42d1-4d5b-8c92-07b894ff00ac'],
 'embeddings': None,
 'documents': ['소득세법\n[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)), 044-215-4312\n기획재정부(소득세제과(근로소득)), 044-215-4216\n기획재정부(금융세제과(이자소득, 배당소득)), 044-215-4233\n기획재정부(소득세제과(사업소득, 기타소득)), 044-215-4217\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n1조(목적)\n이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]',
  '1조의2(정의)\n1항 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n1호 “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 

#### 벡터 DB 수정하기

In [ ]:
# 문서 추가하기
from langchain_core.documents import Document

# page_content, metadata, id 지정
db.add_documents(
    [
        Document(
            page_content="안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요",
            metadata={"source": "mydata.txt"},
            id="1",
        )
    ]
)

In [ ]:
# 문서 삭제하기
# id 1 삭제
db.delete(ids=["1"])

#### 검색기 / Retrival 구현

* 관련 문서는 다음을 참조
  * https://python.langchain.com/docs/integrations/vectorstores/chroma/#initialization  

In [75]:
# mmr 알고리즘 기반 retriever 만들기
mmr_rt = db.as_retriever(
  search_type="mmr",
  search_kwargs={"k": 5, "lambda_mult": 0.25, "fetch_k": 10}
)

In [45]:
for i in mmr_rt.invoke("주식 투자 세금에 대해 알려줘"):
    print(i.page_content)
    print('-------------------------------')

2조의3(신탁재산 귀속소득에 대한 납세의무의 범위)
1항 신탁재산에 귀속되는 소득은 수익자(수익자 사망 시 상속인)에게 귀속되는 것으로 본다.
2항 위탁자가 신탁재산을 실질적으로 통제하는 신탁의 경우 그 소득은 위탁자에게 귀속된다.
[본조신설 2020. 12. 29.]

3조(과세소득의 범위)
1항 거주자에게는 이 법에서 규정하는 모든 소득에 대해 과세한다. 단, 일정한 외국인 거주자는 국외소득 중 국내에서 지급되거나 송금된 소득에만 과세한다.
2항 비거주자에게는 국내원천소득만 과세한다.
3항 일정한 동업자에게는 배분·분배된 초과 소득에 대해 과세한다.
[전문개정 2009. 12. 31.]
-------------------------------


In [51]:
# 유사도 기반 retriever 만들기
sim_rt = db.as_retriever(
  search_type="similarity_score_threshold",
  search_kwargs={"k": 3, "score_threshold": -0.5}
)

for i in sim_rt.invoke("세율 들에 대해 알려줘"):
    print(i.page_content)
    print('-------------------------------')

소득세법
[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]

기획재정부(재산세제과(양도소득세)), 044-215-4312
기획재정부(소득세제과(근로소득)), 044-215-4216
기획재정부(금융세제과(이자소득, 배당소득)), 044-215-4233
기획재정부(소득세제과(사업소득, 기타소득)), 044-215-4217

제1장 총칙 <개정 2009. 12. 31.>

1조(목적)
이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.
[본조신설 2009. 12. 31.]
[종전 제1조는 제2조로 이동 <2009. 12. 31.>]
-------------------------------
2조(납세의무)
1항 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.
1호 거주자
2호 비거주자로서 국내원천소득(국내에서 발생한 소득)이 있는 개인
2항 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.
1호 거주자
2호 비거주자
3호 내국법인
4호 외국법인의 국내지점 또는 국내영업소
5호 그 밖에 이 법에서 정하는 원천징수의무자
3항 「국세기본법」 제13조제1항에 따른 법인 아닌 단체 중 같은 조 제4항에 따른 법인으로 보는 단체 외의 법인 아닌 단체는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 경우에는 1거주자로, 그 밖의 경우에는 1비거주자로 본다.
단, 다음 각 호에 해당하는 경우에는 소득구분에 따라 해당 단체의 각 구성원별로 이 법 또는 「법인세법」에 따라 소득세 또는 법인세를 납부할 의무를 진다.
-------------------------------
8조(상속 등의 경우의 납세지)
1항 상속인 등이 신고한 장소
2항 비거주자가 납세관리인 두면 신고장소
3항 신고 있으면 신고한 장소
4항 신고 없으

c:\Users\JS\.conda\envs\ai\lib\site-packages\langchain_core\vectorstores\base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='dc74cc90-830e-49a8-99e2-db5061ce4e3b', metadata={'source': '..\\ref\\docs\\law_test.txt'}, page_content='소득세법\n[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)), 044-215-4312\n기획재정부(소득세제과(근로소득)), 044-215-4216\n기획재정부(금융세제과(이자소득, 배당소득)), 044-215-4233\n기획재정부(소득세제과(사업소득, 기타소득)), 044-215-4217\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n1조(목적)\n이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]'), -0.2030448701041745), (Document(id='64588749-8408-411e-9460-20b451016385', metadata={'source': '..\\ref\\docs\\law_test.txt'}, page_content='2조(납세의무)\n1항 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n1호 거주자\n2호 비거주자로서 국내원천소득(국내에서 발생한 소득)이 있는 개인\n2항 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n1호 거주자\n2호

#### 멀티 모달 검색(추후 구현)

* 여러 양식의 데이터를 포함하고 쿼리할 수 있는 컬렉션을 지원

#### 이미지를 컬렉션에 포함하기 (이게 필요할까?)

## LLM 체인 연결

In [72]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo",
                 temperature=0)

스트림 출력

In [81]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Answer the following question based on the context: {context}\nQuestion: {input}"
)

chain = create_retrieval_chain(
    retriever=mmr_rt,
    combine_docs_chain=prompt | llm
)

# for chunk in chain.stream({"input": "소득세법 적용 대상에 대해 알려줘"}):
#     print(chunk)

for chunk in chain.stream({"input": "소득세법 적용 대상에 대해 알려줘"}):
    # answer 키가 있을 때만 출력 (context 등은 무시)
    if "answer" in chunk:
        print(chunk["answer"].content, end="", flush=True)

소득세법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 합니다. 이 법은 국내에 주소를 두거나 183일 이상의 거소를 둔 개인인 "거주자"와 거주자가 아닌 개인인 "비거주자"를 대상으로 적용됩니다. 또한, 사업소득이 있는 거주자인 "사업자"도 소득세법의 적용 대상입니다.

In [82]:
response = chain.invoke({"input": "소득세법 적용 대상에 대해 알려줘"})
print(response["answer"].content)

소득세법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 합니다. 이 법은 거주자와 비거주자를 구분하여 적용하며, 거주자는 국내에 주소를 두거나 183일 이상의 거소를 둔 개인을 말하고, 비거주자는 거주자가 아닌 개인을 말합니다. 또한, 사업자는 사업소득이 있는 거주자를 말합니다. 해당 법은 대통령령에 따라 구체적인 주소 및 거주자, 비거주자의 구분이 정해지고 있습니다.


## RAG 평가 파이프라인

In [ ]:

documents = load_documents(pdf_path)
llm = ChatOpenAI(model_name=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0.9)
questions = []

for i in range(num_questions):
    prompt = f"""
다음 문서를 바탕으로 질문을 1개 생성해 주세요.
반드시 질문문장만 출력해 주세요. '질문:'이라는 표현 없이 완전한 한국어 질문 형태로만 작성해 주세요.

문서 내용:
{documents[i % len(documents)].page_content}
"""
    question = llm.invoke(prompt).content
    questions.append(question)